# Processing Tables
codes to clean data received from QGIS

In [ ]:
import pandas as pd
import os
import pyomo.environ as pyo 
import numpy  as np
import matplotlib.pyplot as plt
from datetime import datetime
import gurobipy as gp
from gurobipy import GRB
import seaborn as sns

Data_path_GDrive = "G:/Il mio Drive/Thesis_Large_Files/Working Table"
Data_path_Production = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Production"
Data_path_Consumption = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption"
Data_path_Users = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Users"
results_dir = 'C:/Users/Nik/Documents/GitHub/Thesis/Results/Results_Binary/Final Results'
results_dir = os.path.join(results_dir, datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + '_BinaryProduction')
os.makedirs(results_dir, exist_ok=True)
plot_dir = os.path.join(results_dir,'Plot')
yearly_plot_dir = os.path.join(plot_dir,'Yearly_Plot')
weekly_plot_dir = os.path.join(plot_dir,'Weekly_Plot')
os.makedirs(yearly_plot_dir, exist_ok=True)
os.makedirs(weekly_plot_dir, exist_ok=True)


### Create a uniform datetime format

In [ ]:
# Load DataFrames
DF_name = "2019_ProductionEast.csv"
DF_path = os.path.join(Data_path_Production, DF_name)
DF_East = pd.read_csv(DF_path, sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])

DF_name = "2019_ProductionSouth.csv"
DF_path = os.path.join(Data_path_Production, DF_name)
DF_South = pd.read_csv(DF_path, sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])

DF_name = "2019_ProductionWest.csv"
DF_path = os.path.join(Data_path_Production, DF_name)
DF_West = pd.read_csv(DF_path, sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])

DF_name = "2019_ConsumptionCurve.csv"
DF_path = os.path.join(Data_path_Consumption, DF_name)
DF_Consumption = pd.read_csv(DF_path, sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])

DF_name = "Thesis_User_Residential.csv"
DF_path = os.path.join(Data_path_Users, DF_name)
DF_m2_Residential = pd.read_csv(DF_path,sep=",",on_bad_lines='skip', header=0, parse_dates=[0])

DF_name = "Thesis_User_Industrial.csv"
DF_path = os.path.join(Data_path_Users, DF_name)
DF_m2_Industrial = pd.read_csv(DF_path,sep=",",on_bad_lines='skip', header=0, parse_dates=[0])

DF_name = "Thesis_User_Commercial.csv"
DF_path = os.path.join(Data_path_Users, DF_name)
DF_m2_Commercial = pd.read_csv(DF_path,sep=",",on_bad_lines='skip', header=0, parse_dates=[0])

# List of your dataframes
dataframes = [DF_South, DF_East, DF_West]

# Function to change the datetime index format
def change_datetime_index_format(df):
    df.index = df.index.astype(str)
    df.index = pd.to_datetime(df.index, format='%Y%m%d:%H%M')
    df.index = df.index.strftime('%Y%m%d %H:%M')
    df.index = pd.to_datetime(df.index, format='%Y%m%d %H:%M')
    return df

# Apply the function to each dataframe
dataframes = [change_datetime_index_format(df) for df in dataframes]

# Unpack the dataframes back to their original names if needed
DF_South, DF_East, DF_West = dataframes

# Extract the G(i) column from each DataFrame and rename it
DF_South_Gi = DF_South[['G(i)']].rename(columns={'G(i)': 'G(i)_South'})
DF_East_Gi = DF_East[['G(i)']].rename(columns={'G(i)': 'G(i)_East'})
DF_West_Gi = DF_West[['G(i)']].rename(columns={'G(i)': 'G(i)_West'})

# Combine the columns into a new DataFrame
DF_Production = pd.concat([DF_South_Gi, DF_East_Gi, DF_West_Gi], axis=1)

# Define paths to save the files
DF_Production_name_csv = "2019_Production.csv"
DF_Production_name_xlsx = "2019_Production.xlsx"
DF_path_csv = os.path.join(Data_path_Production, DF_Production_name_csv)
DF_path_xlsx = os.path.join(Data_path_Production, DF_Production_name_xlsx)

# Save the new DataFrame to CSV and Excel
DF_Production.to_csv(DF_path_csv, index=True)
DF_Production.to_excel(DF_path_xlsx, index=True)

# Save the updated DF_Consumption back to CSV with the modified index
DF_Consumption_name_csv = "2019_ConsumptionCurve.csv"
DF_path_consumption_csv = os.path.join(Data_path_Consumption, DF_Consumption_name_csv)
DF_Consumption.to_csv(DF_path_consumption_csv, index=True)

#### Adjust date format to a convenient one

In [ ]:
import os
import pandas as pd

def adjust_date_format_for_datetime_index(folder_path):
    """
    Adjusts the date format of CSV files in the given folder path that have a datetime index,
    by rounding the index to the nearest hour.

    Parameters:
    folder_path (str): The path to the folder containing CSV files.

    Returns:
    None
    """
    # Loop through all files in the folder
    for filename in os.listdir(folder_path):
        # Check if the file is a CSV file
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            
            # Read the CSV file
            try:
                # Load the first few rows to check the index
                df = pd.read_csv(file_path, index_col=0)
                
                # Check if the index is already in datetime format or can be converted
                try:
                    # Try to convert the index to datetime format
                    datetime_index = pd.to_datetime(df.index, errors='raise')
                    
                    # If successful, set the converted index and proceed to round it
                    df.index = datetime_index
                    
                    # Round the index to the nearest hour
                    df.index = df.index.round('H')

                    # Save the adjusted DataFrame back to the original file
                    df.to_csv(file_path)

                    print(f"Processed and saved: {file_path}")

                except Exception as datetime_error:
                    # If the index cannot be converted to datetime, skip the file
                    print(f"Skipped {file_path}: Index is not a valid datetime format. Error: {datetime_error}")

            except Exception as e:
                print(f"Failed to process {file_path}: {e}")

# Run the function with your specified directory
folder_directory = r'C:\Users\Nik\Documents\GitHub\Thesis\CSV\Production'
adjust_date_format_for_datetime_index(folder_directory)


### Evaluating the eligible users to for the PV installation

In [ ]:
# Data from paper for available roof production Area
DoublePitched = 0.5 * 0.6 * 0.85 * 0.7 * 0.5
Industrial = 0.8 * 0.9 * 1 * 0.45 * 1 * 0.45 * 1
Flat = 1 * 0.6 * 0.9 * 0.45 * 0.5
costheta_Res = np.cos(20 * np.pi / 180)
costheta_Ind = np.cos(30 * np.pi / 180)
costheta_Com = np.cos(0 * np.pi / 180)
CoefRes = (DoublePitched * 0.95 + Flat * 0.05) / costheta_Res
CoefInd = Industrial / costheta_Ind
CoefCom = (Flat * 0.5 + DoublePitched * 0.5) / costheta_Com
Limit = 2
nPVLimit = 1.63 * Limit

#Change Area from string to float
DF_m2_Residential['Area'] = DF_m2_Residential['Area'].astype(str)
DF_m2_Industrial['Area'] = DF_m2_Industrial['Area'].astype(str)
DF_m2_Commercial['Area'] = DF_m2_Commercial['Area'].astype(str)

DF_m2_Residential['Area'] = DF_m2_Residential['Area'].str.replace(',', '.')
DF_m2_Industrial['Area'] = DF_m2_Industrial['Area'].str.replace(',', '.')
DF_m2_Commercial['Area'] = DF_m2_Commercial['Area'].str.replace(',', '.')

DF_m2_Residential['Area'] = DF_m2_Residential['Area'].astype(float)
DF_m2_Industrial['Area'] = DF_m2_Industrial['Area'].astype(float)
DF_m2_Commercial['Area'] = DF_m2_Commercial['Area'].astype(float)
DF_m2_Residential['Area'] = DF_m2_Residential['Area'] * CoefRes
DF_m2_Industrial['Area'] = DF_m2_Industrial['Area'] * CoefInd
DF_m2_Commercial['Area'] = DF_m2_Commercial['Area'] * CoefCom

# Delete rows where 'Area' column values are less than nPVLimit
DF_m2_Residential = DF_m2_Residential[DF_m2_Residential['Area'] >= nPVLimit]
DF_m2_Industrial = DF_m2_Industrial[DF_m2_Industrial['Area'] >= nPVLimit]
DF_m2_Commercial = DF_m2_Commercial[DF_m2_Commercial['Area'] >= nPVLimit]

# Reset the index
DF_m2_Residential = DF_m2_Residential.reset_index(drop=True)
DF_m2_Industrial = DF_m2_Industrial.reset_index(drop=True)
DF_m2_Commercial = DF_m2_Commercial.reset_index(drop=True)

#### Generate randomly the exposure coefficients of the users' houses

In [ ]:
# Function to generate random alpha values with two decimal places
def generate_alpha_columns(df):
    def round_and_adjust(row):
        row = np.round(row, 2)
        row_diff = 1.0 - row.sum()
        for i in range(len(row)):
            if row_diff == 0:
                break
            adjustment = np.round(min(row_diff, 0.01 if row_diff > 0 else -0.01), 2)
            if (row[i] + adjustment >= 0) and (row[i] + adjustment <= 1):
                row[i] += adjustment
                row_diff -= adjustment
        return row

    # Generate random values
    random_values = np.random.rand(len(df), 3)
    # Normalize the values to ensure they sum to 1
    random_values /= random_values.sum(axis=1)[:, None]
    # Round values to two decimal places and adjust to ensure they sum to 1
    adjusted_values = np.apply_along_axis(round_and_adjust, 1, random_values)
    # Create new columns
    df['S_coeff'] = adjusted_values[:, 0]
    df['W_coeff'] = adjusted_values[:, 1]
    df['E_coeff'] = adjusted_values[:, 2]
    return df

# Assuming DF_m2_Residential, DF_m2_Industrial, DF_m2_Commercial are already defined
dataframes = [DF_m2_Residential, DF_m2_Industrial, DF_m2_Commercial]

# Apply the function to each dataframe
dataframes = [generate_alpha_columns(df) for df in dataframes]

# Unpack the dataframes back to their original names if needed
DF_m2_Residential, DF_m2_Industrial, DF_m2_Commercial = dataframes

#Save
DF_m2_Residential.to_csv(os.path.join(results_dir, 'DF_m2_Residential.csv'), index=False)
DF_m2_Industrial.to_csv(os.path.join(results_dir, 'DF_m2_Industrial.csv'), index=False)
DF_m2_Commercial.to_csv(os.path.join(results_dir, 'DF_m2_Commercial.csv'), index=False)

# Calculate the production for each residential user
Residential_user_production = (DF_Production['G(i)_South'].values[:, None] * DF_m2_Residential['S_coeff'].values +
                               DF_Production['G(i)_East'].values[:, None] * DF_m2_Residential['E_coeff'].values +
                               DF_Production['G(i)_West'].values[:, None] * DF_m2_Residential['W_coeff'].values)

# Create a DataFrame for residential user production
Residential_user_production_df = pd.DataFrame(Residential_user_production, index=DF_Production.index, columns=DF_m2_Residential.index)

# Calculate the production for each industrial user
Industrial_user_production = (DF_Production['G(i)_South'].values[:, None] * DF_m2_Industrial['S_coeff'].values +
                              DF_Production['G(i)_East'].values[:, None] * DF_m2_Industrial['E_coeff'].values +
                              DF_Production['G(i)_West'].values[:, None] * DF_m2_Industrial['W_coeff'].values)

# Create a DataFrame for industrial user production
Industrial_user_production_df = pd.DataFrame(Industrial_user_production, index=DF_Production.index, columns=DF_m2_Industrial.index)

# Calculate the production for each commercial user
Commercial_user_production = (DF_Production['G(i)_South'].values[:, None] * DF_m2_Commercial['S_coeff'].values +
                              DF_Production['G(i)_East'].values[:, None] * DF_m2_Commercial['E_coeff'].values +
                              DF_Production['G(i)_West'].values[:, None] * DF_m2_Commercial['W_coeff'].values)

# Create a DataFrame for commercial user production
Commercial_user_production_df = pd.DataFrame(Commercial_user_production, index=DF_Production.index, columns=DF_m2_Commercial.index)

# Save the user production DataFrames to CSV files
Residential_user_production_df.to_csv(os.path.join(results_dir, 'Residential_user_production.csv'))
Industrial_user_production_df.to_csv(os.path.join(results_dir, 'Industrial_user_production.csv'))
Commercial_user_production_df.to_csv(os.path.join(results_dir, 'Commercial_user_production.csv'))

# Check the result (optional)
print(Residential_user_production_df.head())
print(Industrial_user_production_df.head())
print(Commercial_user_production_df.head())


In [ ]:
'''# Ensure time column is set as the index and converted to datetime format
DF_Production['time'] = pd.to_datetime(DF_Production['time'])
DF_Production.set_index('time', inplace=True)'''

# Calculate the production for each residential user
Residential_user_production = (
    DF_Production['G(i)_South'].values[:, None] * DF_m2_Residential['S_coeff'].values +
    DF_Production['G(i)_East'].values[:, None] * DF_m2_Residential['E_coeff'].values +
    DF_Production['G(i)_West'].values[:, None] * DF_m2_Residential['W_coeff'].values
)

# Create a DataFrame for residential user production
Residential_user_production_df = pd.DataFrame(
    Residential_user_production,
    index=DF_Production.index,
    columns=[f'Residential_User_{full_id}_Coeff' for full_id in DF_m2_Residential['full_id']]
)

# Similarly, create DataFrames for industrial and commercial user production
Industrial_user_production = (
    DF_Production['G(i)_South'].values[:, None] * DF_m2_Industrial['S_coeff'].values +
    DF_Production['G(i)_East'].values[:, None] * DF_m2_Industrial['E_coeff'].values +
    DF_Production['G(i)_West'].values[:, None] * DF_m2_Industrial['W_coeff'].values
)
Industrial_user_production_df = pd.DataFrame(
    Industrial_user_production,
    index=DF_Production.index,
    columns=[f'Industrial_User_{full_id}_Coeff' for full_id in DF_m2_Industrial['full_id']]
)

Commercial_user_production = (
    DF_Production['G(i)_South'].values[:, None] * DF_m2_Commercial['S_coeff'].values +
    DF_Production['G(i)_East'].values[:, None] * DF_m2_Commercial['E_coeff'].values +
    DF_Production['G(i)_West'].values[:, None] * DF_m2_Commercial['W_coeff'].values
)
Commercial_user_production_df = pd.DataFrame(
    Commercial_user_production,
    index=DF_Production.index,
    columns=[f'Commercial_User_{full_id}_Coeff' for full_id in DF_m2_Commercial['full_id']]
)

# Save the user production DataFrames to CSV files
Residential_user_production_df.to_csv(os.path.join(Data_path_GDrive, 'Residential_user_production.csv'))
Industrial_user_production_df.to_csv(os.path.join(Data_path_GDrive, 'Industrial_user_production.csv'))
Commercial_user_production_df.to_csv(os.path.join(Data_path_GDrive, 'Commercial_user_production.csv'))


# Function to rename columns by removing '*_User_' and '_Mixed'

In [14]:
import pandas as pd

def rename_columns_and_save(input_file_path, output_file_path):
    # Read the CSV file
    df = pd.read_csv(input_file_path)
    
    # Rename the columns
    df.columns = [col.replace('Residential_User_', '').replace('_Mixed', '') if 'Residential_User_' in col else col for col in df.columns]
    
    # Save the modified DataFrame to a new CSV file
    df.to_csv(output_file_path, index=False)
    print(f"File saved as {output_file_path}")


input_file_path = r'C:\Users\Nik\Desktop\Backup thesis\Consumption\Residential_Consumption_Optimized_From_Switched_2.csv'
output_file_path = r'C:\Users\Nik\Desktop\Backup thesis\Consumption\Residential_Consumption_FromBackup.csv'
rename_columns_and_save(input_file_path, output_file_path)


File saved as C:\Users\Nik\Desktop\Backup thesis\Consumption\Residential_Consumption_FromBackup.csv


# Function to modify the time in the datetime index to set minutes to 00


In [15]:
def modify_time_to_hour(dataframe):
    # Convert the index to datetime format (if it's not already) and floor the minute to 'H' (hours)
    dataframe.index = pd.to_datetime(dataframe.index).floor('H')
    return dataframe

# Example usage:
df = pd.read_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\Residential_Consumption_FromBackup.csv', index_col=0)  # Ensure you load your file with the first column as index
df_modified = modify_time_to_hour(df)

# Save the modified dataframe to a new CSV file
df_modified.to_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\Residential_Consumption_FromBackupFinal.csv', index=True)


# Delete columns to decrease the size of the file

In [3]:
import pandas as pd
import random

# Load the data files
residential_consumption_path = r'C:\Users\Nik\Desktop\Backup thesis\Consumption\Residential_Consumption_FromBackupFinal.csv'
residential_production_path = r'G:\Il mio Drive\Thesis_Large_Files\Working Table\Residential_user_production.csv'
df_m2_residential_path = r'C:\Users\Nik\Documents\GitHub\Thesis\CSV\Working Tables\DF_m2_Residential.csv'

# Load the data into DataFrames
residential_consumption_df = pd.read_csv(residential_consumption_path)
residential_production_df = pd.read_csv(residential_production_path)
df_m2_residential_df = pd.read_csv(df_m2_residential_path)

# Select 4000 random columns from residential_consumption_df
random_columns = random.sample(list(residential_consumption_df.columns), 4000)

# Find the full_ids associated with these columns
full_ids_to_delete = set(random_columns).intersection(df_m2_residential_df['full_id'])

# Drop the selected columns in both consumption and production data
residential_consumption_df.drop(columns=full_ids_to_delete, inplace=True, errors='ignore')
residential_production_df.drop(columns=full_ids_to_delete, inplace=True, errors='ignore')

# Remove rows in df_m2_residential_df where full_id is in the list of columns to delete
df_m2_residential_df = df_m2_residential_df[~df_m2_residential_df['full_id'].isin(full_ids_to_delete)]

# Save the modified dataframes back to CSV
residential_consumption_df.to_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_Residential_Consumption.csv', index=False)
residential_production_df.to_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_Residential_Production.csv', index=False)
df_m2_residential_df.to_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_DF_m2_Residential.csv', index=False)

print("Random columns and corresponding rows have been successfully deleted.")


Random columns and corresponding rows have been successfully deleted.
